# Text to SQL Test

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT",""),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name    = os.getenv('DEPLOYMENT_NAME')

In [2]:
user_msg = """### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### Make a query to list the names of the departments which employed more than 10 employees in the last 3 months.
query:
"""

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
  temperature=0,
  max_tokens=500
)

print(response.choices[0].message.content)

To list the names of the departments that employed more than 10 employees in the last 3 months, you can use the following SQL query:

```sql
SELECT d.name AS department_name
FROM Department d
JOIN Employee e ON d.id = e.department_id
WHERE e.id IN (
    SELECT DISTINCT employee_id
    FROM Salary_Payments
    WHERE date >= CURRENT_DATE - INTERVAL '3 months'
)
GROUP BY d.id, d.name
HAVING COUNT(e.id) > 10;
```

### Explanation:
1. **`Salary_Payments` Subquery**:
   - Filters employees who received salary payments in the last 3 months (`date >= CURRENT_DATE - INTERVAL '3 months'`).
   - Uses `DISTINCT` to ensure we only consider unique employees.

2. **Join with `Employee` and `Department`**:
   - Joins the `Employee` table with the `Department` table to associate employees with their respective departments.

3. **Group and Count**:
   - Groups the results by department (`GROUP BY d.id, d.name`).
   - Uses `HAVING COUNT(e.id) > 10` to filter departments with more than 10 employees.

4. *

# SQL to Text Test

In [3]:
user_msg = """###Can you explain what does this code do? 
# Code: 
SELECT d.name
FROM Department d
WHERE d.id IN (
    SELECT e.department_id
    FROM Employee e
    WHERE e.id IN (
        SELECT sp.employee_id
        FROM Salary_Payments sp
        WHERE sp.date >= current_date - interval '3 months'
    )
    GROUP BY e.department_id
    HAVING COUNT(e.id) > 10
)
# Answer: 
"""
   
response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_msg},
    ],
   max_tokens=250)

print(response.choices[0].message.content)

This SQL code is analyzing and querying data from three tables: `Department`, `Employee`, and `Salary_Payments`. Here's a breakdown of what the code does:

### Goal:
The query retrieves the `name` of **departments** (`Department.d.name`) that meet the following criteria:
- **At least 10 employees** in the department have received a **salary payment** in the **last 3 months**.
  
### Breakdown of the Query:
#### 1. **Main query:**
```sql
SELECT d.name
FROM Department d
WHERE d.id IN (
    ...
)
```
- This part fetches the names of departments (`d.name`) from the `Department` table.
- The condition is that the department's `id` must match the results of the **subquery** inside the `WHERE` clause.

---

#### 2. **First subquery:**
```sql
SELECT e.department_id
FROM Employee e
WHERE e.id IN (
    ...
)
GROUP BY e.department_id
HAVING COUNT(e.id) > 10
```
- This subquery works with the `Employee` table.
- It groups employees (`e`) by their `department_id` and checks the condition
